In [ ]:
# !pip install --upgrade pandas
# !pip install --upgrade --ignore-installed PyYAML
# !pip install --upgrade pip
# !pip install --upgrade gen3 --user --upgrade
# !pip install cdiserrors
# !pip install --upgrade pydicom

In [ ]:
import pandas as pd
import sys, os, webbrowser
import gen3
import pydicom
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
import os
# from dicom_csv import join_tree
import subprocess
import zipfile




from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query

In [ ]:
# # Import some custom Python scripts from personal GitHub repo.
# # Change these directory paths to reflect your local working directory.

# home_dir = "/Users/hanchen" 
# demo_dir = "{}/Downloads/Research Working/Federal Learning".format(home_dir)

# print(demo_dir)
# os.chdir(demo_dir)

# os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O {}/expansion.py".format(demo_dir))
# %run expansion.py


In [ ]:
# Initiate instances of the Gen3 SDK Classes using credentials file for authentication.
# Change the directory path in "cred" to reflect the location of your credentials file.

# api = "https://data.midrc.org"
# cred = "/Users/hanchen/Downloads/credentials.json"
# auth = Gen3Auth(api, refresh_file=cred) # authentication class
# sub = Gen3Submission(api, auth) # submission class
# query = Gen3Query(auth) # query class
# exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
# exp.get_project_ids()


In [ ]:
# Reading a CSV file from a file path
csv_file_path = os.getcwd()+"/datasets/MIDRC_Imaging_Studies_table_2.csv"
print(csv_file_path)
csv_data = pd.read_csv(csv_file_path, sep=',', header=0)
csv_data


In [ ]:
# object_ids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855']

# output_addr = "/Users/hanchen/Downloads/Research Working/Federal Learning/"


In [ ]:
def dcm_to_png(image_path):
    ds = pydicom.dcmread(image_path)    
    new_image = ds.pixel_array.astype(float)
    scaled_image = np.uint8((np.maximum(new_image, 0) / new_image.max()) * 255.0)    
    final_image = Image.fromarray(scaled_image)
    final_image.save(image_path[:-4]+'.png')

In [ ]:
def unzip_raw_data(case_id):
    image_path = os.getcwd()+"/datasets/MIDRC_Data/{}/".format(case_id)
    # "/Users/hanchen/Downloads/Research Working/Federal Learning/{}/".format(case_id)

    study_uids = os.listdir(image_path)

    print(image_path)
    print(study_uids)

    for study_uid in study_uids:
        if study_uid.startswith('.'):
            continue;
        if not os.path.isdir(image_path+study_uid):
            continue
        file_names = os.listdir(image_path+study_uid)
        # print(file_names)
        for file_name in file_names:
            if file_name[-3:]!='zip':
                continue;
            folder_name = []
            with zipfile.ZipFile(image_path+study_uid+'/'+file_name, 'r') as zip_file:
                zip_file_names = zip_file.namelist()
                for zip_file_name in zip_file_names:
                    # print(zip_file_name)
                    # Record folder name
                    folder_name = zip_file_name[:zip_file_name.index('/')]

                    # Specify the file name or path within the ZIP archive
                    file_to_fetch = zip_file_name
                    # print('file:',zip_file_name)
                    
                    # Extract the file to the desired location
                    zip_file.extract(file_to_fetch, image_path)
                    # print("File extracted successfully.")
                    
                    # print('FROM:',image_path+zip_file_name)
                    # print('TO:  ',image_path+zip_file_name[zip_file_name.index('/')+1:])
                    os.rename(image_path+zip_file_name, image_path+zip_file_name[zip_file_name.index('/')+1:])
            os.rmdir(image_path+folder_name)
    file_names = os.listdir(image_path)
    for file_name in file_names:
        if file_name[-3:]=='dcm':
            dcm_to_png(image_path+file_name)

In [ ]:
for index, row in csv_data.iterrows():
    unzip_raw_data(row[12])
    print(f"unzip {index} successfully");
    # Access row values using column names or indices
    # for i in range(len(row)):
    #     column1_value = row[i]
    #     print(f"Row {index}, Column {i}: {column1_value}")